# Train a SVM on word embeddings generated by BERTweet

This notebook trains an Support Vector Machine on the word embeddings generated by the BERTweet model. These embeddings are used as the input features for the SVM which classifies wether tweets are abusive, offensive or not.

The notebook can be ran locally or on a service like google colab.

In [ ]:
# Install dependencies
!pip uninstall accelerate transformers -y
!pip install -U accelerate>=0.21.0
!pip3 install transformers
!pip3 install datasets
!pip3 install pandas
!pip3 install torch
!pip3 install scikit-learn
!pip3 install numpy
!pip3 install nltk emoji==0.6.0

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 366.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49720 sha256=7e3428c22d9bda1fd65b35a227d16c76ad2878f915cea4ccdc8b998a574bd692
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji


In [ ]:
# Install the BertTweet Repo, needed for the tweet normalizer
from os import path
from os import path
if not path.exists('./BERTweet/'):
  !git clone https://github.com/VinAIResearch/BERTweet/
import sys
sys.path.append('/content/BERTweet')

In [ ]:
# Use BERTweet to normalize all tweets, so that they match the BERTweet style
from TweetNormalizer import normalizeTweet

def normalize_tweet(tweet):
  # In our data usernames are already normalized to @USER so this will not change anything
  # however URLS are already replaced to URL, but BERTweet uses HTTPURL instead of URL
  # so we have to replace these too, the rest is done by normalizeTweet from BERTweet
  tweet['normalized_text'] = normalizeTweet(tweet['text']).replace('URL', 'HTTPURL')
  return tweet

  # return normalizeTweet(tweet_text).replace('URL', 'HTTPURL')

In [ ]:
# Create the tokenizer
from transformers import AutoTokenizer

tokenizer  = AutoTokenizer.from_pretrained('vinai/bertweet-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.91M [00:00<?, ?B/s]

In [ ]:
# Load the data
import pandas as pd

dataset = pd.read_csv("train_data_offensive_abusive_taskC.csv")

# Split data 80 / 20
dataset_len = len(dataset)

# Shuffle the DataFrame to ensure randomness
dataset = dataset.sample(frac=1, random_state=42)

# Normalize all the tweets, the normalized tweet text
# is saved to the `normalized_text` column
dataset = dataset.apply(normalize_tweet, axis=1)

train = dataset.tail(int(dataset_len*0.8))
train_X = train['normalized_text'].tolist()
train_y = train['abusive_offensive_not'].tolist()

test = dataset.head(int(dataset_len*0.2))
test_X = test['normalized_text'].tolist()
test_y = test['abusive_offensive_not'].tolist()


# Tokenize the data
train_X_tokenized = tokenizer(train_X, padding='max_length', truncation=True, return_tensors='pt')
test_X_tokenized = tokenizer(test_X, padding='max_length', truncation=True, return_tensors='pt')

In [ ]:
# Get the embeddings from bertweet
import torch
from transformers import AutoModel
model = AutoModel.from_pretrained('vinai/bertweet-base')

with torch.no_grad():
    train_outputs = model(**train_X_tokenized)

train_X_embeddings = train_outputs.last_hidden_state.mean(dim=1).numpy()

In [ ]:
# Optional: save the embeddings so we can load them later on
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

with open('/content/gdrive/MyDrive/Uni-23-24/Machine Learning Project/Final Assignment/models/bertweet_svm/embeddings/train_X_v1_embeddings.npy', 'wb') as outfile:
  np.save(outfile, train_X_embeddings)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Load the embeddings (which means the previous cells can be skipped)
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

train_X_embeddings = np.load('/content/gdrive/MyDrive/Uni-23-24/Machine Learning Project/Final Assignment/models/bertweet_svm/embeddings/train_X_v1_embeddings.npy')

Mounted at /content/gdrive


In [ ]:
# Train the SVM classifier
from sklearn.svm import SVC

svm_model = SVC(kernel='linear')
svm_model.fit(train_X_embeddings, train_y)

SVC(kernel='linear')

In [ ]:
# Get the embeddings for the test data
with torch.no_grad():
    test_outputs = model(**test_X_tokenized)

test_X_embeddings = test_outputs.last_hidden_state.mean(dim=1).numpy()

In [ ]:
# Optional: save the embeddings so we can load them later on
# Note: the google colab code can be removed when running locally
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

with open('/content/gdrive/MyDrive/Uni-23-24/Machine Learning Project/Final Assignment/models/bertweet_svm/embeddings/test_X_v1_embeddings.npy', 'wb') as outfile:
  np.save(outfile, test_X_embeddings)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Load the embeddings (skip prev. 2 cells)

test_X_embeddings = np.load('/content/gdrive/MyDrive/Uni-23-24/Machine Learning Project/Final Assignment/models/bertweet_svm/embeddings/test_X_v1_embeddings.npy')

In [ ]:
from sklearn.metrics import f1_score

predictions = svm_model.predict(test_X_embeddings)
f1 = f1_score(test_y, predictions, average='macro')
print("F1 Score:", f1)

F1 Score: 0.4784933927431371


In [ ]:
# Save the sklearn SVM model
import pickle

with open('/content/gdrive/MyDrive/Uni-23-24/Machine Learning Project/Final Assignment/models/bertweet_svm/model.pkl', 'wb') as outfile:
  pickle.dump(svm_model, outfile)